# Death rates sandbox

Prompted by some articles in press claiming a new blood test can predict whether you will die in the next 10 years with an accuracy of 82%, I played a bit with death rate data from SSB to see how hard it is to predict
when someone will die, based ONLY on their age.  This provides a baseline prediction.

If you are young, then it is highly _unlikely_ that you will die in the next 10 years.  If you are very old, it is highly _likely_ that you will die in the next 10 years.  

Approach is to generate a population of people, and determine whether or not each member of this population dies in the next 10 years based on SSB's life tables.  Then make 2 simple models:
1. assume the person won't die (baseline)   
1. assume that everyone dies when they reach Norwegian life expectancy of 82.5 years: anyone younger than 77.5 expects to live for 10 more years, anyone older expects to die.

What a cheerful topic!

August 2019


Related articles:
Sources
https://www.nature.com/articles/s41467-019-11311-9
https://www.nature.com/articles/s41467-019-11311-9/tables/1     Details of test set

https://www.thesun.co.uk/news/9766554/blood-test-predict-when-you-will-die/


https://onezero.medium.com/a-new-test-predicts-when-youll-die-give-or-take-a-few-years-2d08147c8ea6 

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd

from pathlib import Path

#DATA_FOLDER = Path(TODO)

import sklearn
from sklearn import metrics


In [2]:
# This cell copied from https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels



def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
#plot_confusion_matrix(y_test, y_pred, classes=class_names,
#                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
#plot_confusion_matrix(y_test, y_pred, classes=class_names, normalize=True,
#                      title='Normalized confusion matrix')

#plt.show()

In [3]:


class LifeTable():
    # class holding cleaned up data from Norwegian SSB
    # source: https://www.ssb.no/en/dode  
    def __init__(self):
        # p_death is probability per thousand
        READ_FROM_LOCAL_FILE = False
        
        if READ_FROM_LOCAL_FILE:
            src = DATA_FOLDER / "ssb-life-tables.xlsx"
        else:
            src = "https://www.ssb.no/eksport/excel?key=379777"
        _df = pd.read_excel(src, usecols=[0,10], names=['age', 'p_death_per_thou'], header=3, skipfooter=1)
        
        new = _df["age"].str.split(" ", n = 1, expand = True) 
        _df['age'] = pd.to_numeric(new[0])
        
        self.data = _df
        self.data['p_death_next_10y_per_thou']=_df['p_death_per_thou'].rolling(10, min_periods=1).sum()
        self.data = self.data.drop(['age'], axis=1)    
        self.data.index.name = 'age'
    

In [4]:
max(life.data.index)

NameError: name 'life' is not defined

In [ ]:
# make a population, and use life tables to determine if they will die or not the next 10 years.
# Could be made more fancy by giving population a similar age profile to actual population (see example of data for that lower down)
N = 10000
# The headline press articles say the subjects were between 18 and 109 years old
# https://www.nature.com/articles/s41467-019-11311-9/tables/1  gives the details of the subjects
AGE_MIN = 18
AGE_MAX = 109   

LIFE_EXPECTANCY = 82.5   # Norway, wikipedia

life = LifeTable()


ages = np.random.randint(low=AGE_MIN, high=AGE_MAX, size=N)

truth = []                   # whether person will die next 10 years, based on mortality stats
pred_baseline = []           # assume the person will NOT die in the next 10 years
pred_simple = []             # assume everyone dies at life expectancy, so if person is 5 years younger than life expectacny
                             # assume they will live

for person_age in list(ages):
    _age = person_age
    
    if _age > max(life.data.index):
        # if person is older than there is mortality data for, assume they are oldest
        _age = max(life.data.index)
    p_die_next_10y = life.data.iloc[_age]['p_death_next_10y_per_thou']/1000
    _truth = (np.random.uniform(low=0.0, high=1.0)<p_die_next_10y)
    _pred_baseline = False
    if person_age < LIFE_EXPECTANCY-5:
        _pred_simple = False
    else:
        _pred_simple = True
    truth.append(_truth)
    pred_baseline.append(_pred_baseline)
    pred_simple.append(_pred_simple)
    #print(f"Sample age:{person_age} p_die{AGE_MAX}_10(*1000):{p_die_next_10y*1000:.2f} die?:{truth}")
    

print(f"accuracy of 'will you die in next 10 years?'  N={N} ages from {AGE_MIN}-{AGE_MAX}")
print(f"baseline:{sklearn.metrics.accuracy_score(truth, pred_baseline, normalize=True, sample_weight=None)}")
print(f"simple rule:{sklearn.metrics.accuracy_score(truth, pred_simple, normalize=True, sample_weight=None)}")
#plot_confusion_matrix(np.array(truth), np.array(pred), normalize=False, classes=np.array(["die", "live"]), title='Confusion matrix (normalized) Baseline model')

# Older stuff

In [ ]:
np.random.uniform(low=0.0, high=1)

In [ ]:
print(f'{DATA_FOLDER / "age-groups.csv"}')

In [ ]:
# https://www.ethnicity-facts-figures.service.gov.uk/uk-population-by-ethnicity/demographics/age-groups/latest#download-the-data

df = pd.read_csv(DATA_FOLDER / "age-groups.csv")

In [ ]:
df.head()

In [ ]:
# http://www.bandolier.org.uk/booth/Risk/dyingage.html

# data values are the X where the chance of dying in the next year are 1 in X 

df2 = pd.read_csv(DATA_FOLDER / "bandolier-death-rates-by-age.csv")
df2 = df2.drop([0,1])
new = df2["age"].str.split("-", n = 1, expand = True) 
df2['from']=pd.to_numeric(new[0])
df2['to']=pd.to_numeric(new[1])


In [ ]:
df2.head()

# Task
Prompted by this article (which was also reported on CNN): 

https://onezero.medium.com/a-new-test-predicts-when-youll-die-give-or-take-a-few-years-2d08147c8ea6
https://www.thesun.co.uk/news/9766554/blood-test-predict-when-you-will-die/

What is the baseline model for predicting whether or not a person will die in the next 10 years?

Related:
https://onezero.medium.com/a-new-test-predicts-when-youll-die-give-or-take-a-few-years-2d08147c8ea6


First look, just take someone's age now and use it to predict how many years they have left to live



In [ ]:

#df = pd.DataFrame(df.row.str.split(' ',1).tolist(),
#                                   columns = ['flips','row'])

In [ ]:
df2

In [ ]:
def get_prob(X, span):
    """
    convert 1 in X death rate (for a particular span) to a probabilit per year
    e.g. 1 in 100 for a 5 year period corresponds to 0.01/5
    """
    return 1.0/X/span

In [ ]:
df3 = pd.DataFrame()
for index, row in df2.iterrows():
    span = row['to']+1-row['from']
    for age_now in range(row['from'], row['to']+1):
        
        df4 = pd.DataFrame([[age_now, get_prob(row['Men'], span), get_prob(row['Women'], span)]],
                           columns = ['age', 'Men', 'Women'])
        df3 = df3.append(df4)


In [ ]:
df3.plot.scatter(x='age', y='Women')

In [ ]:
life = LifeTable()

In [ ]:
life.data